In [1]:
from fastai.vision.all import *

In [2]:
path = untar_data(URLs.MNIST_SAMPLE)

In [3]:
path.ls()

(#3) [Path('/storage/data/mnist_sample/train'),Path('/storage/data/mnist_sample/labels.csv'),Path('/storage/data/mnist_sample/valid')]

In [4]:
train3_folder = (path/'train'/'3').ls() # dirección de las imágenes
train7_folder = (path/'train'/'7').ls()
valid3_folder = (path/'valid'/'3').ls()
valid7_folder = (path/'valid'/'7').ls()

In [5]:
train_7 = [tensor(Image.open(i)) for i in train7_folder] # crea una lista con las imágenes (puedo abrir cualquiera con show_image(train_7[]))
train_3 = [tensor(Image.open(i)) for i in train3_folder] # idem con el contenido de train3_folder

In [6]:
def stack(folder):
    x = torch.stack(folder).float()/255
    return x

In [7]:
ideal_3 = stack(train_3).mean(0) # crea un tensor de rango-2, donde cada pixel el el promedio 'colapsado' de las matrices que formaban el arreglo tridimensional
ideal_7 = stack(train_7).mean(0)
valid_3 = stack([tensor(Image.open(i)) for i in valid3_folder]) # crea un tensor de rango-3 (1010 matrices de 28x28, siendo cada matriz una imágen)
valid_7 = stack([tensor(Image.open(i)) for i in valid7_folder]) # idem para las imágenes del 7. Se tienen así dos sets de validación, uno para el 3 y otro para el 7

In [8]:
train = torch.cat([stack(train_7), stack(train_3)]) # concatena los tensores de rango-3 (en la celda anterior estos dos tensores tridimensionales tenían un promedio aplicado y se habían convertido en uno bidimensional)
valid = torch.cat([valid_7, valid_3]) # concatena los tensores de rango-3
# train tiene shape (12396, 28, 28)
# valid tiene shape (2038, 28, 28)

train_x = train.view(-1, 28*28) # colapsó el tensor tridimensional en uno bidimensional. Las matrices de 28*28 ahora son vectores de largo 786
valid_x = train.view(-1, 28*28) # el set de validación se muestra también como una colección de vectores

train_y = tensor([1]*len(train_3) + [0]*len(train_7)).unsqueeze(1) # en un vector con 6131 celdas en 1 seguidas de 6265 celdas con valor 0. Son los labels de tensor train_x
valid_y = tensor([1]*len(valid3_folder) + [0]*len(valid7_folder)).unsqueeze(1) # básicamente, el vector de labels para valid_x

In [9]:
dset = list(zip(train_x, train_y)) # genera un arreglo de tuplas (train_x,train_y) --es un arreglo donde cada vector tiene su correspondiente etiqueta--
dset_valid = list(zip(valid_x, valid_y)) # idem para el set de validación

In [10]:
def init_parameters(n_in, n_out, bias = True):
    weights = torch.randn((n_in,n_out)).requires_grad_()
    if bias:
        bias = torch.randn((n_out,1)).requires_grad_()
        return weights, bias
    else:
        return weights

In [11]:
def linear1(xb):
    return xb @ weights + bias # el operador @ es el multiplicador de matrices.

In [12]:
def mnist_loss(predictions, targets): # targets es como se llama a los labels. mnist_loss mide la distancia entre las predicciones y los targets
    predictions = predictions.sigmoid() # sigmoid resalta las diferencias, básicamente
    return (predictions-targets).abs().mean() # alternativa: (predictions-target).abs().mean()

In [13]:
# torch.where(targets==1, 1-predictions, predictions).mean()

In [14]:
def metric(prediction, target):
    return (prediction.sigmoid()-target).float().abs().mean()

In [15]:
dls = DataLoader(dset, batch_size=64) # crea lotes de elementos, para procesar más rápido en paralelo, en la GPU
dls_valid = DataLoader(dset_valid, batch_size=64)

In [21]:
weights,bias = init_parameters(784,1,True) # params[0]=weights, parameters[1]=bias. Esta línea es similar a 'weights,bias = init_parameters(784,1,True)'
lr= 1

def train_epoch(model, lr, weights,bias):  
    for x_batch,y_batch in dls:
        pred = model(x_batch)
        loss = mnist_loss(pred,y_batch)
        loss.backward()
        #print(sigmoid(pred))
        print('error rate:',metric(pred,y_batch).item())
        weights.data -= lr * weights.grad.data
        bias.data -= lr * bias.grad.data
        weights.grad.zero_()
        bias.grad.zero_()

In [22]:
def batch_accuracy(xb,yb): # no podía hacer funcionar la función para la métrica
    preds = xb.sigmoid()
    correct = (preds>0.5) == yb
    return correct.float().mean()

In [23]:
def validate_epoch(model):
    accs = [batch_accuracy(model(xb),yb) for xb,yb in dset_valid]
    return round(torch.stack(accs).mean().item(),4)

In [24]:
def metric_epoch(model):
    accs = [metric(model(xb),yb) for xb,yb in dset_valid]
    return (torch.stack(accs).mean().item())

In [25]:
for i in range(5):
    train_epoch(linear1,lr,weights,bias)
    print(metric_epoch(linear1))

error rate: 0.9692590832710266
error rate: 0.9922423958778381
error rate: 0.9599527716636658
error rate: 0.9305238723754883
error rate: 0.9415126442909241
error rate: 0.9419615864753723
error rate: 0.8657779097557068
error rate: 0.7414360046386719
error rate: 0.6309128403663635
error rate: 0.28498533368110657
error rate: 0.19777542352676392
error rate: 0.09526992589235306
error rate: 0.07614932954311371
error rate: 0.12880642712116241
error rate: 0.03223160654306412
error rate: 0.04028403013944626
error rate: 0.06879056990146637
error rate: 0.027376260608434677
error rate: 0.02695290558040142
error rate: 0.0028565647080540657
error rate: 0.010877775028347969
error rate: 0.03589934483170509
error rate: 0.0030719535425305367
error rate: 0.03376053646206856
error rate: 0.002725604921579361
error rate: 0.015701774507761
error rate: 0.06688323616981506
error rate: 0.009562293998897076
error rate: 0.028545018285512924
error rate: 0.00028971675783395767
error rate: 0.016989637166261673
error 